
### Logs
* 30.08: use 2048 for max_len and max_position_embeddings
* 03.09: use 512 instead
* 23.09: 512 pretrained with both datasets in MLM. training with 2048 and try to infer with 512. Pretraining notebook: pretrain1_task_ver3(512_larger dataset)
* 25.09 512 pretrained with both datasets in MLM. training with 1536. Pretraining notebook: pretrain1_task_ver3(512_larger dataset)
* no pretraining
* no pretraining use max_length as 1600, learning rate change to 2e-5
* no pretraining use max_length as 1600, learning rate changed to 4e-6, cosine scheduler
* no pretraining use max_legnth as 1600, learning rate changed back to 2e-6, cosine scheduler
* change weight decay from 0.01 to 1e-4, learning rate changed to 4e-6
* pretraining 1009, weight decay 1e-4, learning reate 4e-6
* pretraining 2109, weight decay 1e-4, learning reate 4e-6
* batch_size 8, eval batch_size 16
* use pretrained model on new generated data

In [1]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install datasets -q
!pip install accelerate -U -q

In [2]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

from datasets import Dataset, disable_progress_bar
import pandas as pd

pdf = pd.read_csv("./input/prompts_train.csv")
sdf = pd.read_csv("./input/summaries_train.csv")

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /microsoft/deberta-v3-large/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fed5c10dbb0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 1b5cbc21-cb52-4b83-911a-f0a9bba4d5b4)')' thrown while requesting HEAD https://huggingface.co/microsoft/deberta-v3-large/resolve/main/tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/miniconda3/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have

In [3]:
from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large")
## use the pretrained model
# from transformers import AutoConfig
# config = AutoConfig.from_pretrained('./input/pretrain/pretrained_model/')
# model = AutoModelForSequenceClassification.from_pretrained('./input/pretrain/pretrained_model', config = config)

## Train

In [4]:
%%writefile train.py

import os
import logging
import warnings
from dataclasses import dataclass, field
from typing import Optional

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    set_seed,
    Trainer,
    TrainingArguments,
    HfArgumentParser,
    DataCollatorWithPadding,
)
from datasets import Dataset, disable_progress_bar
import pandas as pd
import numpy as np

warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['WANDB_PROJECT'] = 'kaggle-commonlit-eval-student-summaries-0810-pretrain-new-data'

disable_progress_bar()

@dataclass
class Config:
    model_name_or_path: Optional[str] = field(
        default="microsoft/deberta-v3-large",
        metadata={"help": "Model name or path"},
    )

    data_dir: Optional[str] = field(
        default="/kaggle/input/commonlit-evaluate-student-summaries",
        metadata={"help": "Data directory"},
    )

    max_seq_length: Optional[int] = field(
        default=1600,
        #default = 512,
        #default = 512,
        metadata={"help": "Max sequence length"},
    )

    add_prompt_question: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt question into input"},
    )

    add_prompt_text: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt text into input"},
    )

    fold: Optional[int] = field(
        default=0,
        metadata={"help": "Fold"},
    )

    num_proc: Optional[int] = field(
        default=4,
        metadata={"help": "Number of processes"},
    )

    dropout: Optional[float] = field(
        default=0.,
        metadata={"help": "Amount of dropout to apply"},
    )
    max_position_embeddings: Optional[int] = field(
        default=1600,
        #default=512,
        #default = 512,
        metadata={"help": "Amount of dropout to apply"},
    )


# Spell auto correction
# from spellchecker import SpellChecker

# def correct_spelling(input_text):
#     print('input_text: ', input_text)
#     # Initialize the spell checker
#     spell = SpellChecker()
#     # Split the input text into words
#     words = input_text.split()
#     # Initialize an empty list to store the corrected words
#     corrected_words = []
#     for word in words:
#         # Check if the word has any punctuation at the end
#         if word[-1].isalpha():
#             # Extract the punctuation
#             punctuation_end = ""
#         else:
#             punctuation_end = word[-1]
#             word = word[:-1]
#         # check if the word has any punctuation at the start
#         if word[0].isalpha():
#            # Extract the punctuation
#             punctuation_start = ""
#         else:
#             punctuation_start = word[0]
#             word = word[1:]
#         # Check the spelling of the word (case insensitive)
#         corrected_word = spell.correction(word.lower())
#         # Preserve the original capitalization
#         if word[0].isupper():
#             corrected_word = corrected_word.capitalize()
#         # Combine the corrected word and punctuation (if any)
#         corrected_word = punctuation_start+corrected_word+punctuation_end
#         # Append the corrected word to the list
#         corrected_words.append(corrected_word)
#     # Join the corrected words back into a single string
#     corrected_text = " ".join(corrected_words)
#     return corrected_text

def tokenize(example, tokenizer, config):
    sep = tokenizer.sep_token

    # if config.add_prompt_question:
    #     text = sep.join(
    #         [example["prompt_question"], example["prompt_text"], example["text"]]
    #     )
    # elif config.add_prompt_text:
    #     text = sep.join([example["prompt_text"], example["text"]])
    # else:
    #     text = example["text"]
    prompt = sep.join([example["prompt_title"], example["prompt_text"], example["prompt_question"]])
    labels = [example["content"], example["wording"]]

    tokenized = tokenizer(
#         prompt,
#         example["text"],
        example['text'],
        prompt,
        padding=False,
        truncation=True, # changed by Peng, turn on the truncation
        max_length=config.max_seq_length,
    )

    return {
        **tokenized,
        "labels": labels,
    }




def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }


def main():
    parser = HfArgumentParser((Config, TrainingArguments))

    config, training_args = parser.parse_args_into_dataclasses()

    set_seed(training_args.seed)

    if "wandb" in training_args.report_to:
        import wandb

        try:
#             from kaggle_secrets import UserSecretsClient
#             user_secrets = UserSecretsClient()
#             key = user_secrets.get_secret("wandb")

#             wandb.login(key=key)
            wandb.login()
        except:
            print("Could not log in to WandB")

    tokenizer = AutoTokenizer.from_pretrained(config.model_name_or_path)
    model_config = AutoConfig.from_pretrained(config.model_name_or_path)

    model_config.update({
        "hidden_dropout_prob": config.dropout,
        "attention_probs_dropout_prob": config.dropout,
        "num_labels": 2,
        "problem_type": "regression",
        "max_position_embeddings": config.max_position_embeddings,
        "cfg": config.__dict__,
    })

    print(model_config)
    
    # Do not use pretrained model
#     print('do not use pretrained model')
#     model = AutoModelForSequenceClassification.from_pretrained(
#         config.model_name_or_path, config=model_config
#     )
    
    # use pretrained model
    print('use pretrained_model')
    print('./pretrain_seed42_0810/')
    model = AutoModelForSequenceClassification.from_pretrained('./pretrain_seed42_0810/', config = model_config)

    #pdf = pd.read_csv(f"{config.data_dir}/prompts_train.csv")
    pdf = pd.read_csv(f"./input/prompts_train.csv")
    #sdf = pd.read_csv(f"{config.data_dir}/summaries_train.csv")
    sdf = pd.read_csv(f"./input/summaries_train.csv")

    df = pdf.merge(sdf, on="prompt_id")

    # 4 prompt ids, 4 folds
    id2fold = {
        "814d6b": 0,
        "39c16e": 1,
        "3b9047": 2,
        "ebad26": 3,
    }

    df["fold"] = df["prompt_id"].map(id2fold)

    train_ds = Dataset.from_pandas(df[df["fold"] != config.fold])
    val_ds = Dataset.from_pandas(df[df["fold"] == config.fold])

    train_ds = train_ds.map(
        tokenize,
        batched=False,
        num_proc=config.num_proc,
        fn_kwargs={"tokenizer": tokenizer, "config": config},
    )

    val_ds = val_ds.map(
        tokenize,
        batched=False,
        num_proc=config.num_proc,
        fn_kwargs={"tokenizer": tokenizer, "config": config},
    )

    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        pad_to_multiple_of=16 if training_args.fp16 else None,
    )

    training_args.bf16 =True
    training_args.gradient_accumulation_steps = 1
    training_args.load_best_model_at_end = True
    training_args.greater_is_better = False
    training_args.metric_for_best_model = 'mcrmse'
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_mcrmse,
    )

    trainer.train()

    model.config.best_metric = trainer.state.best_metric
    model.config.save_pretrained(training_args.output_dir)

    trainer.log({"eval_best_mcrmse": trainer.state.best_metric})


if __name__ == "__main__":
    main()

Overwriting train.py


In [6]:
from pathlib import Path

seed = 42

fold = 1

output = f"output_fold{fold}_seed{seed}_0810"

!python train.py \
  --model_name_or_path "microsoft/deberta-v3-large" \
  --add_prompt_question True \
  --fold $fold \
  --data_dir "./" \
  --output_dir $output \
  --fp16 \
  --num_train_epochs 4 \
  --dataloader_num_workers 4 \
  --learning_rate 6e-6 \
  --weight_decay 1e-4 \
  --warmup_ratio 0 \
  --lr_scheduler_type "cosine" \
  --optim "adamw_torch" \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2 \
  --evaluation_strategy "steps" \
  --eval_steps 150 \
  --save_strategy "steps" \
  --save_steps 150 \
  --save_total_limit 1 \
  --report_to "wandb" \
  --metric_for_best_model "mcrmse" \
  --greater_is_better False \
  --logging_steps 10 \
  --log_level "error" \
  --disable_tqdm True \
  --ddp_find_unused_parameters False \
  --dropout 0 \
  --seed $seed


output_dir = Path.cwd() / output
# add json files
for json_file in output_dir.glob("checkpoint*/*token*.json"):
    json_file.rename(output_dir/json_file.name)

# model files
for model_file in output_dir.glob("checkpoint*/*model*"):
    model_file.rename(output_dir/model_file.name)

# remove optimizer states and other files
to_delete = str(list(output_dir.glob("checkpoint*"))[0])
!rm -r $to_delete

wandb: Currently logged in as: peng_sun. Use `wandb login --relogin` to force relogin
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_probs_dropout_prob": 0.0,
  "cfg": {
    "add_prompt_question": true,
    "add_prompt_text": false,
    "data_dir": "./",
    "dropout": 0.0,
    "fold": 1,
    "max_position_embeddings": 1600,
    "max_seq_length": 1600,
    "model_name_or_path": "microsoft/deberta-v3-large",
    "num_proc": 4
  },
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 1600,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
 

{'loss': 0.3727, 'learning_rate': 5.925339924771599e-06, 'epoch': 0.29}
{'loss': 0.2761, 'learning_rate': 5.923280735666507e-06, 'epoch': 0.29}
{'loss': 0.304, 'learning_rate': 5.921193902098344e-06, 'epoch': 0.29}
{'eval_loss': 0.3712873160839081, 'eval_content_rmse': 0.4283958077430725, 'eval_wording_rmse': 0.7476981282234192, 'eval_mcrmse': 0.5880469679832458, 'eval_runtime': 112.0766, 'eval_samples_per_second': 18.354, 'eval_steps_per_second': 9.181, 'epoch': 0.29}
{'loss': 0.3315, 'learning_rate': 5.919079443801577e-06, 'epoch': 0.3}
{'loss': 0.1863, 'learning_rate': 5.916937380771914e-06, 'epoch': 0.3}
{'loss': 0.2772, 'learning_rate': 5.91476773326611e-06, 'epoch': 0.31}
{'loss': 0.3627, 'learning_rate': 5.912570521801777e-06, 'epoch': 0.31}
{'loss': 0.2059, 'learning_rate': 5.9103457671571895e-06, 'epoch': 0.31}
{'loss': 0.4711, 'learning_rate': 5.9080934903710895e-06, 'epoch': 0.32}
{'loss': 0.2658, 'learning_rate': 5.905813712742487e-06, 'epoch': 0.32}
{'loss': 0.3864, 'learn

{'eval_loss': 0.35587766766548157, 'eval_content_rmse': 0.37540891766548157, 'eval_wording_rmse': 0.7555286288261414, 'eval_mcrmse': 0.5654687881469727, 'eval_runtime': 112.1864, 'eval_samples_per_second': 18.336, 'eval_steps_per_second': 9.172, 'epoch': 0.65}
{'loss': 0.2792, 'learning_rate': 5.618884243032701e-06, 'epoch': 0.65}
{'loss': 0.3264, 'learning_rate': 5.614371763878002e-06, 'epoch': 0.65}
{'loss': 0.2354, 'learning_rate': 5.609834561506404e-06, 'epoch': 0.66}
{'loss': 0.3414, 'learning_rate': 5.6052726788246735e-06, 'epoch': 0.66}
{'loss': 0.2836, 'learning_rate': 5.60068615897297e-06, 'epoch': 0.67}
{'loss': 0.2092, 'learning_rate': 5.5960750453244395e-06, 'epoch': 0.67}
{'loss': 0.1651, 'learning_rate': 5.5914393814848e-06, 'epoch': 0.67}
{'loss': 0.534, 'learning_rate': 5.586779211291935e-06, 'epoch': 0.68}
{'loss': 0.3126, 'learning_rate': 5.582094578815476e-06, 'epoch': 0.68}
{'loss': 0.1825, 'learning_rate': 5.577385528356383e-06, 'epoch': 0.69}
{'loss': 0.3357, 'lea

{'loss': 0.2028, 'learning_rate': 5.120015258716376e-06, 'epoch': 1.0}
{'loss': 0.1543, 'learning_rate': 5.11347781542074e-06, 'epoch': 1.01}
{'loss': 0.1407, 'learning_rate': 5.10692038569044e-06, 'epoch': 1.01}
{'loss': 0.1606, 'learning_rate': 5.100343031536843e-06, 'epoch': 1.01}
{'loss': 0.1734, 'learning_rate': 5.093745815159729e-06, 'epoch': 1.02}
{'loss': 0.1771, 'learning_rate': 5.0871287989467105e-06, 'epoch': 1.02}
{'loss': 0.1767, 'learning_rate': 5.080492045472639e-06, 'epoch': 1.03}
{'loss': 0.2259, 'learning_rate': 5.0738356174990154e-06, 'epoch': 1.03}
{'loss': 0.1525, 'learning_rate': 5.067159577973397e-06, 'epoch': 1.03}
{'loss': 0.1982, 'learning_rate': 5.060463990028798e-06, 'epoch': 1.04}
{'loss': 0.1891, 'learning_rate': 5.0537489169830965e-06, 'epoch': 1.04}
{'loss': 0.2159, 'learning_rate': 5.0470144223384355e-06, 'epoch': 1.05}
{'loss': 0.1737, 'learning_rate': 5.040260569780621e-06, 'epoch': 1.05}
{'loss': 0.317, 'learning_rate': 5.033487423178516e-06, 'epoch'

{'loss': 0.1165, 'learning_rate': 4.435096294183581e-06, 'epoch': 1.37}
{'loss': 0.12, 'learning_rate': 4.426988042081848e-06, 'epoch': 1.37}
{'loss': 0.1998, 'learning_rate': 4.418866295443735e-06, 'epoch': 1.37}
{'loss': 0.2564, 'learning_rate': 4.410731131073813e-06, 'epoch': 1.38}
{'loss': 0.1238, 'learning_rate': 4.402582625903544e-06, 'epoch': 1.38}
{'loss': 0.2053, 'learning_rate': 4.394420856990544e-06, 'epoch': 1.39}
{'loss': 0.186, 'learning_rate': 4.3862459015178615e-06, 'epoch': 1.39}
{'loss': 0.1503, 'learning_rate': 4.378057836793246e-06, 'epoch': 1.39}
{'loss': 0.1469, 'learning_rate': 4.3698567402484175e-06, 'epoch': 1.4}
{'loss': 0.3682, 'learning_rate': 4.361642689438332e-06, 'epoch': 1.4}
{'loss': 0.1515, 'learning_rate': 4.353415762040451e-06, 'epoch': 1.41}
{'loss': 0.3381, 'learning_rate': 4.345176035854003e-06, 'epoch': 1.41}
{'eval_loss': 0.22165200114250183, 'eval_content_rmse': 0.3825652301311493, 'eval_wording_rmse': 0.5449292659759521, 'eval_mcrmse': 0.46374

...

{'loss': 0.0694, 'learning_rate': 1.9627740995078986e-06, 'epoch': 2.45}
{'loss': 0.0825, 'learning_rate': 1.9541224509283333e-06, 'epoch': 2.45}
{'loss': 0.1455, 'learning_rate': 1.9454806928539563e-06, 'epoch': 2.46}
{'loss': 0.0829, 'learning_rate': 1.9368489070069096e-06, 'epoch': 2.46}
{'loss': 0.112, 'learning_rate': 1.9282271750150306e-06, 'epoch': 2.47}
{'eval_loss': 0.26208576560020447, 'eval_content_rmse': 0.3677981197834015, 'eval_wording_rmse': 0.6236152052879333, 'eval_mcrmse': 0.4957066774368286, 'eval_runtime': 112.1509, 'eval_samples_per_second': 18.341, 'eval_steps_per_second': 9.175, 'epoch': 2.47}
{'loss': 0.1095, 'learning_rate': 1.91961557841108e-06, 'epoch': 2.47}
{'loss': 0.1035, 'learning_rate': 1.9110141986319747e-06, 'epoch': 2.47}
{'loss': 0.1257, 'learning_rate': 1.9024231170180101e-06, 'epoch': 2.48}
{'loss': 0.1255, 'learning_rate': 1.8938424148121e-06, 'epoch': 2.48}
{'loss': 0.0805, 'learning_rate': 1.885272173159e-06, 'epoch': 2.49}
{'loss': 0.1131, 'le

{'loss': 0.1947, 'learning_rate': 1.2189282808991153e-06, 'epoch': 2.81}
{'loss': 0.0817, 'learning_rate': 1.211513009286295e-06, 'epoch': 2.82}
{'loss': 0.0927, 'learning_rate': 1.2041146507813983e-06, 'epoch': 2.82}
{'eval_loss': 0.24191154539585114, 'eval_content_rmse': 0.3739357888698578, 'eval_wording_rmse': 0.5865105390548706, 'eval_mcrmse': 0.4802231788635254, 'eval_runtime': 112.2099, 'eval_samples_per_second': 18.332, 'eval_steps_per_second': 9.17, 'epoch': 2.82}
{'loss': 0.0879, 'learning_rate': 1.1967332753481613e-06, 'epoch': 2.82}
{'loss': 0.0487, 'learning_rate': 1.1893689527897212e-06, 'epoch': 2.83}
{'loss': 0.1012, 'learning_rate': 1.1820217527479487e-06, 'epoch': 2.83}
{'loss': 0.1032, 'learning_rate': 1.174691744702795e-06, 'epoch': 2.83}
{'loss': 0.0786, 'learning_rate': 1.1673789979716306e-06, 'epoch': 2.84}
{'loss': 0.0856, 'learning_rate': 1.1600835817085934e-06, 'epoch': 2.84}
{'loss': 0.0945, 'learning_rate': 1.1528055649039333e-06, 'epoch': 2.85}
{'loss': 0.10

{'eval_loss': 0.2519169747829437, 'eval_content_rmse': 0.3805117905139923, 'eval_wording_rmse': 0.5992034673690796, 'eval_mcrmse': 0.48985761404037476, 'eval_runtime': 111.9457, 'eval_samples_per_second': 18.375, 'eval_steps_per_second': 9.192, 'epoch': 3.17}
{'loss': 0.0899, 'learning_rate': 6.102392060634592e-07, 'epoch': 3.18}
{'loss': 0.0423, 'learning_rate': 6.046734740717571e-07, 'epoch': 3.18}
{'loss': 0.0486, 'learning_rate': 5.991303938612838e-07, 'epoch': 3.18}
{'loss': 0.0689, 'learning_rate': 5.936100178510456e-07, 'epoch': 3.19}
{'loss': 0.056, 'learning_rate': 5.881123982453463e-07, 'epoch': 3.19}
{'loss': 0.0494, 'learning_rate': 5.826375870332877e-07, 'epoch': 3.19}
{'loss': 0.0494, 'learning_rate': 5.771856359882797e-07, 'epoch': 3.2}
{'loss': 0.0507, 'learning_rate': 5.717565966675558e-07, 'epoch': 3.2}
{'loss': 0.0788, 'learning_rate': 5.663505204116763e-07, 'epoch': 3.21}
{'loss': 0.0645, 'learning_rate': 5.609674583440516e-07, 'epoch': 3.21}
{'loss': 0.0967, 'learn

{'loss': 0.053, 'learning_rate': 2.0596869153198018e-07, 'epoch': 3.53}
{'loss': 0.0562, 'learning_rate': 2.0262252111794355e-07, 'epoch': 3.53}
{'loss': 0.0865, 'learning_rate': 1.9930280454307014e-07, 'epoch': 3.54}
{'loss': 0.0445, 'learning_rate': 1.9600957320078316e-07, 'epoch': 3.54}
{'loss': 0.062, 'learning_rate': 1.927428582340408e-07, 'epoch': 3.54}
{'loss': 0.0544, 'learning_rate': 1.8950269053504487e-07, 'epoch': 3.55}
{'loss': 0.0338, 'learning_rate': 1.8628910074495097e-07, 'epoch': 3.55}
{'loss': 0.0403, 'learning_rate': 1.8310211925357446e-07, 'epoch': 3.56}
{'loss': 0.082, 'learning_rate': 1.799417761991069e-07, 'epoch': 3.56}
{'loss': 0.0584, 'learning_rate': 1.7680810146782678e-07, 'epoch': 3.56}
{'loss': 0.0691, 'learning_rate': 1.7370112469382327e-07, 'epoch': 3.57}
{'loss': 0.1937, 'learning_rate': 1.706208752587085e-07, 'epoch': 3.57}
{'loss': 0.025, 'learning_rate': 1.6756738229134417e-07, 'epoch': 3.57}
{'loss': 0.0438, 'learning_rate': 1.6454067466756729e-07, 

{'loss': 0.0517, 'learning_rate': 1.208657092324339e-08, 'epoch': 3.89}
{'loss': 0.0579, 'learning_rate': 1.127340906794816e-08, 'epoch': 3.89}
{'loss': 0.06, 'learning_rate': 1.0488510574726661e-08, 'epoch': 3.9}
{'loss': 0.0858, 'learning_rate': 9.731882866095165e-09, 'epoch': 3.9}
{'loss': 0.03, 'learning_rate': 9.003533097221794e-09, 'epoch': 3.9}
{'loss': 0.0424, 'learning_rate': 8.303468155861249e-09, 'epoch': 3.91}
{'loss': 0.0635, 'learning_rate': 7.63169466228686e-09, 'epoch': 3.91}
{'loss': 0.0572, 'learning_rate': 6.988218969230298e-09, 'epoch': 3.92}
{'loss': 0.0646, 'learning_rate': 6.3730471618203005e-09, 'epoch': 3.92}
{'loss': 0.0537, 'learning_rate': 5.786185057526039e-09, 'epoch': 3.92}
{'loss': 0.0769, 'learning_rate': 5.227638206100504e-09, 'epoch': 3.93}
{'loss': 0.0986, 'learning_rate': 4.6974118895302124e-09, 'epoch': 3.93}
{'loss': 0.0831, 'learning_rate': 4.195511121983575e-09, 'epoch': 3.94}
{'eval_loss': 0.25359031558036804, 'eval_content_rmse': 0.38209727406

In [10]:
from pathlib import Path
import json

seed = 42
scores = []
for fold in range(0,4):
    output = f"/root/autodl-tmp/output_fold{fold}_seed{seed}_0810"
    p = Path(output) / "config.json"

    with open(p) as fp:
        cfg = json.load(fp)


    print(f"MCRMSE for fold {fold}, seed {seed}: {cfg['best_metric']:.4f}")
    scores.append(cfg['best_metric'])
print(sum(scores)/len(scores))

MCRMSE for fold 0, seed 42: 0.5363
MCRMSE for fold 1, seed 42: 0.4454
MCRMSE for fold 2, seed 42: 0.4974
MCRMSE for fold 3, seed 42: 0.4314
0.47761109471321106


#df.loc[0, 'prompt_text']

In [18]:
# from pathlib import Path
# import json

# seed = 42
# scores = []
# for fold in range(4):
#     output = f"./output_fold{fold}_seed{seed}_3008"
#     p = Path(output) / "config.json"

#     with open(p) as fp:
#         cfg = json.load(fp)


#     print(f"MCRMSE for fold {fold}, seed {seed}: {cfg['best_metric']:.4f}")
#     scores.append(cfg['best_metric'])
# print(sum(scores)/len(scores))